# Wizeline Data Engineering Bootcamp Challenge

## Considerations

Given that the input file's size is ~20 GB, my first approach was to use the [dask](https://github.com/dask/dask) library. `dask` is a `pandas` wrapper that supports slicing big data so it can be processed in machines with "not so much" memory.

However, I faced many issues when trying to do transformations with the data.

So, after reading documentation for a bit, I discovered that using only the necessary columns for analysis helps a lot during processing.

Reading the data in chunks also seems like a common practice when dealing with large datasets.

**Note**: I'll be using this notebook to deliver my report.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Just taking a look to the sample data to get an idea of how the information could be useful.

In [2]:
import pandas as pd

In [3]:
sample_df = pd.read_csv('sample.csv')
sample_df.head()

,producto,presentacion,marca,categoria,catalogo,precio,fechaRegistro,cadenaComercial,giro,nombreComercial,direccion,estado,municipio,latitud,longitud
0,CUADERNO FORMA ITALIANA,96 HOJAS PASTA DURA. CUADRICULA CHICA,ESTRELLA,MATERIAL ESCOLAR,UTILES ESCOLARES,25.9,2011-05-18 00:00:00.000,ABASTECEDORA LUMEN,PAPELERIAS,ABASTECEDORA LUMEN SUCURSAL VILLA COAPA,CANNES No. 6 ESQ. CANAL DE MIRAMONTES,DISTRITO FEDERAL,TLALPAN,19.29699,-99.125417
1,CRAYONES,CAJA 12 CERAS. JUMBO. C.B. 201423,CRAYOLA,MATERIAL ESCOLAR,UTILES ESCOLARES,27.5,2011-05-18 00:00:00.000,ABASTECEDORA LUMEN,PAPELERIAS,ABASTECEDORA LUMEN SUCURSAL VILLA COAPA,CANNES No. 6 ESQ. CANAL DE MIRAMONTES,DISTRITO FEDERAL,TLALPAN,19.29699,-99.125417
2,CRAYONES,CAJA 12 CERAS. TAMANO REGULAR C.B. 201034,CRAYOLA,MATERIAL ESCOLAR,UTILES ESCOLARES,13.9,2011-05-18 00:00:00.000,ABASTECEDORA LUMEN,PAPELERIAS,ABASTECEDORA LUMEN SUCURSAL VILLA COAPA,CANNES No. 6 ESQ. CANAL DE MIRAMONTES,DISTRITO FEDERAL,TLALPAN,19.29699,-99.125417
3,COLORES DE MADERA,CAJA 12 PIEZAS LARGO. TRIANGULAR. C.B. 640646,PINCELIN,MATERIAL ESCOLAR,UTILES ESCOLARES,46.9,2011-05-18 00:00:00.000,ABASTECEDORA LUMEN,PAPELERIAS,ABASTECEDORA LUMEN SUCURSAL VILLA COAPA,CANNES No. 6 ESQ. CANAL DE MIRAMONTES,DISTRITO FEDERAL,TLALPAN,19.29699,-99.125417
4,COLOR LARGO,CAJA 36 PIEZAS. CON SACAPUNTAS. 68-4036,CRAYOLA,MATERIAL ESCOLAR,UTILES ESCOLARES,115.0,2011-05-18 00:00:00.000,ABASTECEDORA LUMEN,PAPELERIAS,ABASTECEDORA LUMEN SUCURSAL VILLA COAPA,CANNES No. 6 ESQ. CANAL DE MIRAMONTES,DISTRITO FEDERAL,TLALPAN,19.29699,-99.125417


`dask` had some issues with .zip files, but fortunately, `pandas` doesn't have that problem, so I don't have to upload this huge file to my Drive.

Before loading the dataset, with a bit of help of the sample dataset, we could infer the columns that could be useful to answer the required questions.

In [4]:
useful_cols = ['producto', 'cadenaComercial', 'estado', 'fechaRegistro']

**Note**: if you want to reproduce the results, please create a shortcut in your Drive from the original file and update the filepath in the `read_csv` function.

In [5]:
df = pd.read_csv('/content/drive/MyDrive/WizeDataEngineering/profeco.zip', usecols=useful_cols)

## 1. How many commercial chains are monitored, and therefore, included in this database?

There are 705 commercial chains in this database. To get this number, I used the `value_counts` function in the `cadenaComercial` pd.Series. I also was curious if there was any missing data, but the results are the same before and after the `dropna` execution.

In [6]:
df['cadenaComercial'].value_counts()

WAL-MART                           8643133
BODEGA AURRERA                     6765453
SORIANA                            6546211
MEGA COMERCIAL MEXICANA            4899509
CHEDRAUI                           4221625
                                    ...   
FARMACIA PROVIDENCIA                     7
DANIEL S UNIFORMES                       5
MUEBLERIA MI CASA                        5
PESCADERIA EXPENDIO LOS RUFINOS          4
PESCADERIA LA AMISTAD                    1
Name: cadenaComercial, Length: 705, dtype: int64

In [7]:
df['cadenaComercial'].dropna(inplace=True)

In [8]:
df['cadenaComercial'].value_counts()

WAL-MART                           8643133
BODEGA AURRERA                     6765453
SORIANA                            6546211
MEGA COMERCIAL MEXICANA            4899509
CHEDRAUI                           4221625
                                    ...   
FARMACIA PROVIDENCIA                     7
DANIEL S UNIFORMES                       5
MUEBLERIA MI CASA                        5
PESCADERIA EXPENDIO LOS RUFINOS          4
PESCADERIA LA AMISTAD                    1
Name: cadenaComercial, Length: 705, dtype: int64

## 2. What are the top 10 monitored products by State?

The answer can be found in the `states_info.csv` file.

In [9]:
df['producto'].dropna(inplace=True)
df['estado'].dropna(inplace=True)

In [10]:
df['estado'].value_counts()

DISTRITO FEDERAL                   11284102
MÉXICO                              8173302
JALISCO                             4552128
NUEVO LEÓN                          3171091
GUANAJUATO                          2638456
YUCATÁN                             2300994
MICHOACÁN DE OCAMPO                 2093037
TLAXCALA                            2081024
QUINTANA ROO                        2076525
PUEBLA                              2021476
TABASCO                             1842633
SINALOA                             1720736
SONORA                              1698620
QUERÉTARO                           1667824
COAHUILA DE ZARAGOZA                1512473
ZACATECAS                           1383201
BAJA CALIFORNIA                     1200999
TAMAULIPAS                          1170493
COLIMA                              1159974
OAXACA                              1075420
HIDALGO                             1017667
BAJA CALIFORNIA SUR                  977128
CHIHUAHUA                       

After executing `value_counts` once, I found there were some records with only the "estado" string, I'm removing them here.

In [11]:
df = df[df['estado'] != 'estado']

In [12]:
df['estado'].value_counts()

DISTRITO FEDERAL                   11284102
MÉXICO                              8173302
JALISCO                             4552128
NUEVO LEÓN                          3171091
GUANAJUATO                          2638456
YUCATÁN                             2300994
MICHOACÁN DE OCAMPO                 2093037
TLAXCALA                            2081024
QUINTANA ROO                        2076525
PUEBLA                              2021476
TABASCO                             1842633
SINALOA                             1720736
SONORA                              1698620
QUERÉTARO                           1667824
COAHUILA DE ZARAGOZA                1512473
ZACATECAS                           1383201
BAJA CALIFORNIA                     1200999
TAMAULIPAS                          1170493
COLIMA                              1159974
OAXACA                              1075420
HIDALGO                             1017667
BAJA CALIFORNIA SUR                  977128
CHIHUAHUA                       

In [13]:
top_products_by_state = pd.concat([
  df[df['estado'] == state].groupby(['estado', 'producto'])['producto'].count().nlargest(10)
  for state in df['estado'].unique()                                  
])

top_products_by_state.to_csv('states_info.csv')

# We can take a look to the first 15 items by state.
# For the full list, go to the file created in the previous line.
top_products_by_state.head(15)

estado            producto               
DISTRITO FEDERAL  REFRESCO                   287463
                  FUD                        207569
                  LECHE ULTRAPASTEURIZADA    175640
                  DETERGENTE P/ROPA          173452
                  YOGHURT                    136720
                  CERVEZA                    136686
                  MAYONESA                   131103
                  CHILES EN LATA             130598
                  JABON DE TOCADOR           129889
                  SHAMPOO                    125603
MÉXICO            REFRESCO                   194939
                  FUD                        149141
                  DETERGENTE P/ROPA          132862
                  LECHE ULTRAPASTEURIZADA    116522
                  JABON DE TOCADOR            97330
Name: producto, dtype: int64

## 3. Which is the commercial chain with the highest number of monitored products?

It's Wal-Mart with 8643133 products.

In [14]:
highest_number_monitored_products = df.groupby('cadenaComercial')['producto'].count().nlargest(1)
highest_number_monitored_products

cadenaComercial
WAL-MART    8643133
Name: producto, dtype: int64

## 4. Use the data to find an interesting fact.

I was curious to see what the most sold product was per year.

In [15]:
df['fechaRegistro'] = pd.to_datetime(df['fechaRegistro'], errors='coerce')
df['fechaRegistro'].dropna(inplace=True)
df['year'] = df['fechaRegistro'].dt.year.astype('Int64')

top_product_by_year = pd.concat([
  df[df['year'] == year].groupby(['year', 'producto'])['producto'].count().nlargest(1)
  for year in range(df['year'].min(), df['year'].max())
])
top_product_by_year

year  producto         
2011  DETERGENTE P/ROPA    155874
2012  DETERGENTE P/ROPA    167878
2013  REFRESCO             182415
2014  REFRESCO             282878
2015  REFRESCO             370494
Name: producto, dtype: int64

## 5. What are the lessons learned from this exercise?

Definitely that managing large amounts of data's not trivial task. One must find creative ways in how to process the information when the resources are not optimal. For example, we wouldn't have to worry about `groupby` operations if we had infinite memory at our disposal, instead we need to figure out how to handle such operations so we can experiment with the hypothesis we formulate.

## 6. Can you identify other ways to approach this problem? Explain.

I was thinking that maybe all the processing could be done by a data warehouse, probably something like BigQuery, so that we only need to worry about how to send the queries to it and get the results backed by the huge infrastructure it has behind.